In [1]:
import numpy as np
import socket
from PIL import Image
from keras.models import load_model
%matplotlib inline

width = 64
height = 64
channel = 3

Using TensorFlow backend.


## Load the models

In [2]:
lidar_model = load_model('car_model_lidar.h5')
camera_model = load_model('car_model_camera.h5')

## Class for environment & agent interactions

In [3]:
class EnvironmentInteraction:
    
    def __init__(self, Address, Port):
        
        # This byte array like delimeter uses to divide input byte socket stream to camera and lidar image
        self.byteDelimeter = b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4 + b'\x00' * 4 + b'\xff' * 4
        
        # The ip address of Unity client, usually set to null
        self.address = Address
        
        # The port to listen
        self.port = Port
        
        # The socket object
        self.sock = socket.socket()
        
        # Init socket object
        self.sock.bind((self.address, self.port))
            
        
    def listen(self):
        
        # Start to listen
        self.sock.listen(1)
        
        # Store connection when get request
        self.connection, self.address = self.sock.accept()
        
        
    def step(self, action):
        
        # Send the action to environment
        self.connection.send(action.encode('utf-8'))                        # Send action to environment
        
        # Receive the next images
        data = self.connection.recv(768432)                                 # Get camera and lidar images as byte array
        
        # Split to two byte arrays
        data = data.split(self.byteDelimeter)
        
        # Save the byte arrays as images
        camera = Image.frombytes('RGB', (64, 64), data[0])
        lidar = Image.frombytes('RGB', (64, 64), data[1])
        
        # Convert to numpy ndarray with 64x64x3 shape to flip images
        camera = np.array(camera.getdata(), dtype = np.float32).reshape(width, height, channel)
        lidar = np.array(lidar.getdata(), dtype = np.float32).reshape(width, height, channel)
        
        # Normalize the data
        camera = camera / 255
        lidar = lidar / 255
        
        # Flip the images horizontally
        camera = np.flip(camera, axis = 0)    
        lidar = np.flip(lidar, axis = 0)
        
        # Convert to numpy ndarray with 1x64x64x3 shape to predict by models
        camera = np.array(camera, dtype = np.float32).reshape(1, width, height, channel)
        lidar = np.array(lidar, dtype = np.float32).reshape(1, width, height, channel)
                
        return camera, lidar

# Object of EnvironmentInteraction class
environment = EnvironmentInteraction('', 7777) 

In [4]:
lidar_target_names = ['Human', 'Clear path', 'Obstacle']
lane_target_names = ['Left Lane', 'Right Lane']
counter = 0
obstacleDetected = False

# Function for formatting the command to environment
# The control of car in the environment makes by only one number, angle of turning
def command(lane_output, angle_output, lidar_output):
    
    global lidar_target_names
    global lane_target_names
    global counter
    global obstacleDetected
    
    action = '0'
    
    
    if counter == 15:
        counter = 0
        obstacleDetected = False
    
    # This condition is uses to give the car time to go over the next lane 
    if ((obstacleDetected) & (lidar_target_names[lidar_output] != 'Obstacle')):
        counter += 1
        return '0'
    
    # The main priority gives to lidar model, rather than camera model
    # Depending on what object is detected, system will make decision how to control the car
    if lidar_target_names[lidar_output] == 'Human':
        action = '-999'     # Stop moving
    elif lidar_target_names[lidar_output] == 'Obstacle':
        obstacleDetected = True
        if lane_target_names[lane_output] == 'Left Lane':
            action = '45'   # move to right lane
        else:
            action = '-45'  # move to left lane
    else:
        action = str(angle_output)
    
    return action

In [ ]:
environment.listen()
print(environment.connection)

action = '0#_#_'

while(True):
        
    # Send the action and get next images to predict
    cameraImage, lidarImage = environment.step(action)
    
    # Make predictions
    camera_output = camera_model.predict(cameraImage)
    lidar_output = lidar_model.predict(lidarImage)
    
    # Calculate turning angle
    angle = camera_output[1][0][0] * 60
    
    # Form a new command
    action = command(np.argmax(camera_output[0]), angle, np.argmax(lidar_output)) + '#' + lane_target_names[np.argmax(camera_output[0])] + '#' + lidar_target_names[np.argmax(lidar_output)]
    
    print(action)